In [21]:
### Importeer bibliotheken en utils

import os
import pandas as pd
import json
from IPython.display import JSON as JSONDisplay
import requests
from re import sub
import ast
import re

import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, Sequence
from sqlalchemy.dialects.postgresql import insert

import uuid

def generateEAUUID():
    return '{' +  str(uuid.uuid4()).upper() + '}'

# Wijzig de maximale breedte van kolommen (bijvoorbeeld naar 100 tekens)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_rows', 20) 

# Controleer of de instelling is toegepast
print(pd.get_option('max_colwidth'))

generateEAUUID()

100


'{F0837572-B82E-4371-89CC-3E9807BF0196}'

In [22]:
### Configuratie
db_uri = "sqlite:////home/jovyan/root/v2.2.0-develop/gemeentelijk gegevensmodel EA16.qea"


In [23]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.automap import automap_base

# Databaseverbinding
engine = create_engine(db_uri)

# Reflecteer metadata
metadata = MetaData()
metadata.reflect(bind=engine)
print("Beschikbare tabellen:", metadata.tables.keys())

# Automap Base instellen
Base = automap_base(metadata=metadata)
Base.prepare(autoload_with=engine)

# Controleer beschikbare klassen
print("Beschikbare klassen:", Base.classes.keys())

# Toegang tot klassen (gebruik de juiste tabelnamen)
try:
    Object = Base.classes.t_object
    XRef = Base.classes.t_xref
    
    print("Klassen geladen succesvol.")
except AttributeError as e:
    print("Fout bij het laden van klassen:", e)
    

Beschikbare tabellen: dict_keys(['sqlite_sequence', 't_attribute', 't_attributeconstraints', 't_attributetag', 't_authors', 't_cardinality', 't_category', 't_clients', 't_complexitytypes', 't_connector', 't_connectorconstraint', 't_connectortag', 't_connectortypes', 't_constants', 't_constrainttypes', 't_datatypes', 't_diagram', 't_diagramlinks', 't_diagramobjects', 't_diagramtypes', 't_document', 't_ecf', 't_efforttypes', 't_files', 't_genopt', 't_glossary', 't_html', 't_image', 't_implement', 't_issues', 't_lists', 't_mainttypes', 't_method', 't_metrictypes', 't_object', 't_objectconstraint', 't_objecteffort', 't_objectfiles', 't_objectmetrics', 't_objectproblems', 't_objectproperties', 't_objectrequires', 't_objectresource', 't_objectrisks', 't_objectscenarios', 't_objecttests', 't_objecttrx', 't_objecttypes', 't_ocf', 't_operation', 't_operationparams', 't_operationposts', 't_operationpres', 't_operationtag', 't_package', 't_palette', 't_paletteitem', 't_phase', 't_primitives', 't_

In [52]:
from sqlalchemy.orm import sessionmaker

# Maak een sessie
Session = sessionmaker(bind=engine)
session = Session()

# Query alle records uit XRef
records = session.query(XRef).all()


# Omzetten naar DataFrame
data = [
    {column: getattr(record, column) for column in record.__table__.columns.keys()}
    for record in records
]
df_xref = pd.DataFrame(data)

# Toon het DataFrame
df_xref[df_xref['Client'].isin(['{440219A4-C64B-4eac-ADE5-E79ED6AA9BFE}', '{5772BEBB-97FA-42a9-B70D-DB55EAD6D1EE}', '{331C4A0B-1505-4945-A0B0-DCD8703AB50F}'])]
df_xref

,XrefID,Name,Type,Visibility,Namespace,Requirement,Constraint,Behavior,Partition,Description,Client,Supplier,Link
0,{56AD8743-771F-4dcd-AE35-C1100E88E20F},CustomProperties,element property,Public,None,None,None,None,None,@PROP=@NAME=isActive@ENDNAME;@TYPE=Boolean@ENDTYPE;@VALU=@ENDVALU;@PRMT=@ENDPRMT;@ENDPROP;,{FF448272-AB9D-4ec9-B4BE-E60E2552817A},None,None
1,{897F21EB-DB59-422a-8D4D-53D01DD22B6F},Stereotypes,attribute property,Public,None,None,None,None,0,@STEREO;Name=enum;GUID={37BCBB77-4979-4330-BDBD-4294FB9C0E11};@ENDSTEREO;,{F0B3E3F9-33FE-4f53-9A7D-5F51F987D782},<none>,None
2,{F025558B-82C0-40ac-A8DE-EC60BBA7B6EA},Stereotypes,attribute property,Public,None,None,None,None,0,@STEREO;Name=enum;GUID={37BCBB77-4979-4330-BDBD-4294FB9C0E11};@ENDSTEREO;,{10314A38-6EE6-4ab8-9265-90E57C8AB71A},<none>,None
3,{BB825F3B-33A5-4417-B9D3-2AD79301B21B},Stereotypes,attribute property,Public,None,None,None,None,0,@STEREO;Name=enum;GUID={37BCBB77-4979-4330-BDBD-4294FB9C0E11};@ENDSTEREO;,{81BB74A4-C279-4556-B166-5525DD490FFA},<none>,None
4,{53FAB5BF-574C-4978-BAD9-E66DEBF55D8E},Stereotypes,attribute property,Public,None,None,None,None,0,@STEREO;Name=enum;GUID={37BCBB77-4979-4330-BDBD-4294FB9C0E11};@ENDSTEREO;,{B76953F7-6713-457c-A327-BF95F7D8EF05},<none>,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6941,{548B559F-2074-4687-8E3C-172E026C308B},Stereotypes,element property,Public,None,None,None,None,0,@STEREO;Name=Attribuutsoort;FQName=MIM::Attribuutsoort;@ENDSTEREO;,{99E0904E-09F4-4119-ACAB-4049FC09439B},None,None
6942,{5E43991D-6FAC-466A-BCE3-014FAD39C983},Stereotypes,element property,Public,None,None,None,None,0,@STEREO;Name=Attribuutsoort;FQName=MIM::Attribuutsoort;@ENDSTEREO;,{dst495D53-B1CF-4dda-A6CA-903B2ECC735F},None,None
6943,{15A50590-8E64-4365-8009-A23232589376},Stereotypes,element property,Public,None,None,None,None,0,@STEREO;Name=Attribuutsoort;FQName=MIM::Attribuutsoort;@ENDSTEREO;,{dst883E50-FEE4-4c26-A82E-28BB87F53C62},None,None
6944,{DC4CCF97-C4BA-4BC4-A4D6-4C390AACD698},Stereotypes,element property,Public,None,None,None,None,0,@STEREO;Name=Attribuutsoort;FQName=MIM::Attribuutsoort;@ENDSTEREO;,{dst6E7F9D-D7F2-4279-B35B-981DCDA389E0},None,None


In [53]:
# Functie om het gewenste formaat te maken
def translate_eaid(value):
    # Verwijder 'EAID_' en vervang '_' door '-'
    core_value = value.replace("EAID_", "").replace("_", "-")
    # Omhul met accolades
    return f"{{{core_value}}}"


df_classes = pd.read_excel("../root/v2.2.0-develop/Gemeentelijk Gegevensmodel.xlsx", sheet_name="classes")
df_classes["ea_guid"] = df_classes["id"].apply(translate_eaid)
df_classes

df_enumerations = pd.read_excel("../root/v2.2.0-develop/Gemeentelijk Gegevensmodel.xlsx", sheet_name="enumerations")
df_enumerations["ea_guid"] = df_enumerations["id"].apply(translate_eaid)
df_enumerations

df_attributes = pd.read_excel("../root/v2.2.0-develop/Gemeentelijk Gegevensmodel.xlsx", sheet_name="attributes")
df_attributes["ea_guid"] = df_attributes["id"].apply(translate_eaid)
df_attributes


,id,bron,clazz_id,created,definitie,enumeration_id,modified,name,primitive,schema_id,stereotype,toelichting,type_class_id,verplicht,ea_guid
0,EAID_8E91C8DE_D513_4907_9180_A44CEED1B1DA,NaN,EAID_F1E55DC7_0F33_40ea_8713_2E1AC3D7EE8D,NaN,NaN,NaN,NaN,aanvangAanwezigheid,DateTime,GGM_ROOT_SCHEMA,NaN,NaN,NaN,False,{8E91C8DE-D513-4907-9180-A44CEED1B1DA}
1,EAID_A5E50423_238E_466b_BE99_907CEBEFC748,NaN,EAID_F1E55DC7_0F33_40ea_8713_2E1AC3D7EE8D,NaN,NaN,NaN,NaN,eindeAanwezigheid,DateTime,GGM_ROOT_SCHEMA,NaN,NaN,NaN,False,{A5E50423-238E-466b-BE99-907CEBEFC748}
2,EAID_085F1605_6837_440f_AA02_0875ADA2C3A0,NaN,EAID_F1E55DC7_0F33_40ea_8713_2E1AC3D7EE8D,NaN,NaN,NaN,NaN,naam,AN250,GGM_ROOT_SCHEMA,NaN,NaN,NaN,False,{085F1605-6837-440f-AA02-0875ADA2C3A0}
3,EAID_87E80E09_D1E1_405a_B523_7B0325C6036C,NaN,EAID_F1E55DC7_0F33_40ea_8713_2E1AC3D7EE8D,NaN,NaN,EAID_E2EDC770_08EB_4298_A2BA_F7943ACD695F,NaN,rol,NaN,GGM_ROOT_SCHEMA,NaN,NaN,NaN,False,{87E80E09-D1E1-405a-B523-7B0325C6036C}
4,EAID_7320C8AA_C6D0_4002_AA34_83215D0387AA,NaN,EAID_F1E55DC7_0F33_40ea_8713_2E1AC3D7EE8D,NaN,NaN,NaN,NaN,vertegenwoordigtOrganisatie,AN250,GGM_ROOT_SCHEMA,NaN,NaN,NaN,False,{7320C8AA-C6D0-4002-AA34-83215D0387AA}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4388,EAID_99E0904E_09F4_4119_ACAB_4049FC09439B,NaN,EAID_ABC4F2A1_A1C6_4db4_9227_4D7D63C5F94E,NaN,Een code die aangeeft welke nationaliteit de ingeschrevene bezit.,NaN,NaN,valutacode,AN3,GGM_ROOT_SCHEMA,Referentiegegeven,NaN,NaN,False,{99E0904E-09F4-4119-ACAB-4049FC09439B}
4389,EAID_dst495D53_B1CF_4dda_A6CA_903B2ECC735F,NaN,EAID_AE553F3F_B8B6_4bd2_A6A1_8051145C823B,NaN,NaN,NaN,NaN,NaN,NaN,GGM_ROOT_SCHEMA,NaN,NaN,EAID_0799B0D1_5067_43e2_8A6F_48FB972225E5,False,{dst495D53-B1CF-4dda-A6CA-903B2ECC735F}
4390,EAID_dst883E50_FEE4_4c26_A82E_28BB87F53C62,NaN,EAID_DE4D6CAB_7798_46cd_AE1B_328684681323,NaN,NaN,NaN,NaN,NaN,NaN,GGM_ROOT_SCHEMA,NaN,NaN,EAID_0799B0D1_5067_43e2_8A6F_48FB972225E5,False,{dst883E50-FEE4-4c26-A82E-28BB87F53C62}
4391,EAID_dst6E7F9D_D7F2_4279_B35B_981DCDA389E0,NaN,EAID_EF4E7B58_F178_4a4f_85C5_A5CD93747A36,NaN,NaN,NaN,NaN,NaN,NaN,GGM_ROOT_SCHEMA,NaN,NaN,EAID_97EE5348_F06A_44cb_A792_7371CF5D2E53,False,{dst6E7F9D-D7F2-4279-B35B-981DCDA389E0}


In [56]:
pd.set_option('display.max_rows', 20)  # Toon alle rijen
def getXrefMerge(df_xref, df):
    # Merge/join op een gemeenschappelijke kolom
    result = pd.merge(df_xref, df, left_on="Client", right_on="ea_guid", how="inner")
    result = result[['name', 'definitie', 'Description', 'ea_guid', 'XrefID']].copy()
    result = result[result.Description.str.startswith('@STEREO')]
    return result


df_xref_classes = getXrefMerge(df_xref, df_classes)
df_xref_classes

df_xref_enumerations = getXrefMerge(df_xref, df_enumerations)
df_xref_enumerations

df_xref_attributes = getXrefMerge(df_xref, df_attributes)
df_xref_attributes[df_xref_attributes.name == 'buurtgeometrie']

,name,definitie,Description,ea_guid,XrefID
113,buurtgeometrie,De tweedimensionale geometrische representatie van de omtrekken van de buurt.,@STEREO;Name=Attribuutsoort;FQName=MIM::Attribuutsoort;@ENDSTEREO;,{8C6F68FD-D6A8-4ad2-BCA2-9CBBD5DED993},{3D666C37-F590-496B-9314-92E120E1436D}


In [50]:
def deleteXRefs(engine, df_xref):
    # Maak een sessie aan
    Session = sessionmaker(bind=engine)
    session = Session()

    # Lijst van XRefID's om te verwijderen
    xref_ids_to_delete = list(df_xref['XrefID'])  # Vervang door je eigen XRefID's

    try:
        # Zoek en verwijder records
        session.query(XRef).filter(XRef.XrefID.in_(xref_ids_to_delete)).delete(synchronize_session=False)
        session.commit()  # Wijzigingen opslaan
        print(f" {len(xref_ids_to_delete)} Records succesvol verwijderd.")
    except Exception as e:
        session.rollback()  # Terugrollen bij een fout
        print(f"Fout bij het verwijderen van records: {e}")
    finally:
        session.close()
        
#deleteXRefs(engine, df_xref_classes)
#deleteXRefs(engine, df_xref_enumerations)
deleteXRefs(engine, df_xref_attributes) # Nog niet uitgevoerd


 988 Records succesvol verwijderd.


In [51]:
class_string = "@STEREO;Name=Objecttype;FQName=MIM::Objecttype;@ENDSTEREO;"
enum_string = "@STEREO;Name=Enumeratie;FQName=MIM::Enumeratie;@ENDSTEREO;"
attr_string = "@STEREO;Name=Attribuutsoort;FQName=MIM::Attribuutsoort;@ENDSTEREO;"

def createXRefs(engine, df_input, string):
    # Maak een sessie
    Session = sessionmaker(bind=engine)
    session = Session()

    # Lijst van Client ID's om nieuwe XRef records te maken
    client_ids = list(df_input['ea_guid'])  

    try:
        # Maak nieuwe XRef records
        new_records = []
        for client_id in client_ids:
            new_record = XRef(
                XrefID=generateEAUUID(),  # Voeg andere vereiste kolommen toe
                Name='Stereotypes',
                Type='element property',
                Visibility='Public',
                Description=string,
                Partition=0,
                Client=client_id  # Kolomnaam moet overeenkomen met de tabel
            )
            new_records.append(new_record)

        # Voeg de records toe aan de sessie
        session.add_all(new_records)
        session.commit()  # Sla de wijzigingen op in de database

        print(f"{len(new_records)} records succesvol toegevoegd aan XRef.")
    except Exception as e:
        session.rollback()  # Herstel de transactie bij fouten
        print(f"Fout bij het toevoegen van records: {e}")
    finally:
        session.close()

#createXRefs(engine, df_classes, class_string)
#createXRefs(engine, df_enumerations, enum_string)
createXRefs(engine, df_attributes, attr_string)  # Nog niet uitgevoerd

4393 records succesvol toegevoegd aan XRef.
